Aim : Create/select a folder for placement of sfv --> download a ytv (will add selecting a lcv).


term list : sfv - Short form video
            ytv - Youtube video
            lcv - local video


Before implementing a content analysis model that uses openai, i am starting with chapters that are provided by the channel

In [1]:
from pytube import YouTube
import moviepy.editor as mpe

import requests
import re
import os


In [2]:
def ytv_downloader(video_link):
    # Specify the folder path, 
    SAVE_PATH = "/home/mali/repurpose/videofolder" # this folder will be accessed in video_chop.py
    ytv = YouTube(video_link)
    
    # Get the highest resolution stream
    #ytv = ytv.streams.get_highest_resolution()


    #ytv_stream = ytv.streams.get_highest_resolution()

    #ytv_stream = ytv.streams.filter(adaptive=True).filter(mime_type='video/webm').first()
    #print(ytv_stream)
    # Download the video to the specified folder
    #print("downloading...")
    #ytv_stream.download(output_path = SAVE_PATH)
    #print("video finished downloading.")

    video_name = f"{ytv.title}_clip.mp4"
    audio_name = f"{ytv.title}_audio.mp3"

    # Download video and rename
    video = ytv.streams.filter(subtype='mp4', res="1080p").first()
    print("downloading video...")
    video_path = video.download(output_path = SAVE_PATH)
    new_video_path = os.path.join(SAVE_PATH, video_name)
    os.rename(video_path, new_video_path)
    print("video finished downloading.")

    # Download audio and rename
    audio = ytv.streams.filter(only_audio=True).first()
    print("downloading audio...")
    audio_path = audio.download(output_path = SAVE_PATH)
    new_audio_path = os.path.join(SAVE_PATH, audio_name)
    os.rename(audio_path, new_audio_path)
    print("audio finished downloading.")

    # Setting the audio to the video
    video = mpe.VideoFileClip(new_video_path)
    audio = mpe.AudioFileClip(new_audio_path)
    final = video.set_audio(audio)

    # Output result
    output_name = f"{ytv.title}_concat.mp4"
    output_path = os.path.join(SAVE_PATH, output_name)
    final.write_videofile(output_path)

    # Delete video and audio to keep the result
    os.remove(new_video_path)
    os.remove(new_audio_path)


    print(f"here is the url {video_link}")
    return video_link

In [3]:
#the only global variable is the link
ytv_url = "https://www.youtube.com/watch?v=phOkyQo7rjo"

ytv_downloader(ytv_url)

downloading video...
video finished downloading.
downloading audio...
audio finished downloading.
Moviepy - Building video /home/mali/repurpose/videofolder/Escape Into Peaceful Autumn Nostalgia | Fall Ambient Music - Rec.5_concat.mp4.
MoviePy - Writing audio in Escape Into Peaceful Autumn Nostalgia | Fall Ambient Music - Rec.5_concatTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /home/mali/repurpose/videofolder/Escape Into Peaceful Autumn Nostalgia | Fall Ambient Music - Rec.5_concat.mp4



Moviepy - Done !
Moviepy - video ready /home/mali/repurpose/videofolder/Escape Into Peaceful Autumn Nostalgia | Fall Ambient Music - Rec.5_concat.mp4
here is the url https://www.youtube.com/watch?v=phOkyQo7rjo


'https://www.youtube.com/watch?v=phOkyQo7rjo'

In [41]:
ytv_test = YouTube(ytv_url)
#mp4_streams = ytv_test.streams.filter(file_extension='mp4').all()
#ytv_test.title

#ytv_test_stream = ytv_test.streams.filter(adaptive=True).filter(mime_type='video/webm').first()
#print(ytv_test_stream)
print(f"{ytv_test.title}_audio.mp3")
audio_name = f"{ytv_test.title}_audio.mp3"
# Download audio and rename
audio = ytv_test.streams.filter(only_audio=True).first()
print("downloading audio...")
audio_file_path = audio.download(output_path = "/home/mali/repurpose/videofolder")
new_audio_file_path = os.path.join("/home/mali/repurpose/videofolder", audio_name)
os.rename(audio_file_path, new_audio_file_path)
print("audio finished downloading.")

Escape Into Peaceful Autumn Nostalgia | Fall Ambient Music - Rec.5_audio.mp3
downloading audio...
audio finished downloading.


In [35]:
print(re.sub("[^a-zA-Z0–9#-_!’”]","_", ytv_test.title))

Escape_Into_Peaceful_Autumn_Nostalgia___Fall_Ambient_Music_-_Rec.5


In [39]:
ytv_test.title

'Escape Into Peaceful Autumn Nostalgia | Fall Ambient Music - Rec.5'

Currently yt.description is not working (08.2024) --> temp solution

In [47]:
#Saved the description in a string variable. want to remove the chapterstamps out into its own list

ytv_link = requests.get("https://www.youtube.com/watch?v=phOkyQo7rjo").text
search = re.search(r'shortDescription":"', ytv_link)
description = ""
count = search.start() + 19  # adding the length of the 'shortDescription":"
while True:
    # get the letter at current index in text
    letter = ytv_link[count]
    if letter == "\"":
        if ytv_link[count - 1] == "\\":
            # this is case where the letter before is a backslash, meaning it is not real end of description
            description += letter
            count += 1
        else:
            break
    else:
        description += letter
        count += 1

print(f'description: {description}')

#Length of video - converted to timestamp format
video_len = yt.length

seconds = video_len % (24 * 3600)
seconds %= 3600
minutes = seconds // 60
seconds %= 60

video_len = ("%02d:%02d" % (minutes, seconds)) # Format --> MM:SS

print(f"length of video {video_len} min")

#extract timestamps
timestamps = re.findall(r"\d+:\d+",description)
print(f"Found {len(timestamps)} timestamps")

description: Take a moment to escape into Spyro the Dragon's enchanting world, where gentle ambient space music wraps around you like a warm blanket. As Spyro soars through autumn skies, the blend of calming sounds and nostalgic visuals brings a sense of peace and comfort, almost like reconnecting with an old friend. It's more than just a game—it's a soothing retreat that invites you to relax, unwind, and lose yourself in the beauty of a beloved classic. CozyCorner is your place to slow down and savor the simple joys.\n\n\nSubscribe and leave a like!!\nShare your feelings in the comments below!\n\nTimestamps⏰\n00:00 \"first light of autumn\"\n04:07 \"harvest breeze\"\n07:07 \"first night of autumn\"\n11:07 \"golden leaves\"\n15:07 \"quaint village\"\n18:12 \"village market stroll\"\n20:20 \"candlelit evening\"\n21:18 \"harvest moon rising\"\n24:30 \"when will the snow fall?\"\n\n#relaxingmusic #darkambience #sleepaid #gamingnostalgia\n#ambientmusic #backgroundmusic #relaxingmusic #medi

Error handling: https://github.com/pytube/pytube/issues/1954

There seems to be an issue with the pytube .download function.

To address this issue i have linked the forum with the answer above.

In [ ]:
video_name = f"{ytv.title}_clip.mp4"
audio_name = f"{ytv.title}_audio.mp3"

# Download video and rename
video = ytv.YouTube(url).streams.filter(subtype='mp4', res="1080p").first()
print("downloading video...")
video.download(output_path = SAVE_PATH)
os.rename(video, video_name)
print("video finished downloading.")

# Download audio and rename
audio = ytv.YouTube(url).streams.filter(only_audio=True).first()
print("downloading audio...")
audio.download(output_path = SAVE_PATH)
os.rename(audio, audio_name)
print("audio finished downloading.")

# Setting the audio to the video
video = mpe.VideoFileClip(video_name)
audio = mpe.AudioFileClip(audio_name)
final = video.set_audio(audio)

# Output result
final.write_videofile(f"{ytv_title}_concat.mp4")

# Delete video and audio to keep the result
os.remove(video_name)
os.remove(audio_name)